In [1]:
import camelot
import openpyxl
path = './4/'

In [20]:
def prepare_df(file_name):
    pages = '10,11'
    
    tables = camelot.read_pdf(path + file_name, pages=pages)
    res = []
    for i in range(0, len(tables)):
        res.append(tables[i].df)
        
    return res

def epd_number(file_name):
    page = '2'
    tables = camelot.read_pdf(path + file_name, pages=page)
    df = tables[0].df
    return df[df.eq('Declaration Number').any(1)][1].tolist()[0]

def get_traci(file_name, header, stage):
    page = '5'
    tables = camelot.read_pdf(path + file_name, pages=page, flavor='stream')
    df = tables[0].df
    print("****** TRACI**********")
    print(header, stage)
    if stage == 'A1-A3':
        return float(df[3][header]) + float(df[4][header])
    if stage in ['A4','A5','B1','B2','B3','B4','B5','B6','B7']: 
        return float(df[5][header])
    if stage in ['C1','C2','C3','C4']:
        return float(df[6][header])
    if stage == 'D':
        return 0 


In [24]:
stages = ['A1-A3', 'A4', 'C1', 'C2', 'C3', 'C4', 'D']

# EXCEL FILE
filename = 'NSF_excel_test.xlsx'
book = openpyxl.Workbook()
book.save(filename)


sheet = book['Sheet']
sheet.cell(row=1, column=1).value = 'Filename'
sheet.cell(row=1, column=2).value = 'EPD_Number'
sheet.cell(row=1, column=3).value = 'DownloadLink'
sheet.cell(row=1, column=4).value = 'Stages'


headers = [
    "impactGWP100_kgCO2e",
    "impactODP_kgCFC11e",
    "impactAP_kgSO2e",
    "impactEP_kgPO4e",
    "impactPOCP_kgEthenee",
    "impactADPElements_kgSbe",
    "impactADPFossilFuels_MJ",
    
    "renewablesUsedAsEnergy_MJ",
    "renewablesUsedAsMaterial_MJ",
    "embodiedRenewableEnergy_MJ",
    "nonRenewablesUsedAsEnergy_MJ",
    "nonRenewablesUsedAsMaterial_MJ",
    "embodiedEnergy_MJ",
    
    "recyclingMaterialUse_kg",
    "renewableRecylingFuelUse_MJ",
    "nonRenewableRecylingFuelUse_MJ",
    "cleanWaterNetUse_m3",
    "wasteHazardous_kg",
    "wasteNonHazardous_kg",
    "wasteRadioactive_kg",
    "reusableMaterialsOutput_kg",
    "recyclableMaterialsOutput_kg",
    "energyMaterialsOutput_kg",
    "exportedEnergyOutput_MJ_thermal",
    "exportedEnergyOutput_MJ_electric",
    "exportedEnergyOutput_MJ",
    
    "traciGWP_kgCO2e",
    "traciODP_kgCFC11e",
    "traciAP_kgSO2e",
    "traciEP_kgNe",
    "traciPOCP_kgO3e",
    "traciNRPE_MJ"
]

columns = {
  "impactGWP100_kgCO2e": "GWP - Total",
  "impactODP_kgCFC11e": "ODP",
  "impactAP_kgSO2e": "AP",
  "impactEP_kgPO4e": "*EP - Freshwater",
  "impactPOCP_kgEthenee": "POCP",
  "impactADPElements_kgSbe": "ADPE",
  "impactADPFossilFuels_MJ": "ADPF",
    
  "renewablesUsedAsEnergy_MJ": "PERE",
  "renewablesUsedAsMaterial_MJ": "PERM",
  "embodiedRenewableEnergy_MJ": "PERT",
  "nonRenewablesUsedAsEnergy_MJ": "PENRE",
  "nonRenewablesUsedAsMaterial_MJ": "PENRM",
  "embodiedEnergy_MJ": "PENRT",
  "recyclingMaterialUse_kg": "SM",
  "renewableRecylingFuelUse_MJ": "RSF",
  "nonRenewableRecylingFuelUse_MJ": "NRSF",
  "cleanWaterNetUse_m3": "FW",
    
  "wasteHazardous_kg": "HWD",
  "wasteNonHazardous_kg": "NHWD",
  "wasteRadioactive_kg": "RWD",
  "reusableMaterialsOutput_kg": "CRU",
  "recyclableMaterialsOutput_kg": "MFR",
  "energyMaterialsOutput_kg": "MER",
  "exportedEnergyOutput_MJ_thermal": "EE (Thermal)",
  "exportedEnergyOutput_MJ_electric": "EE (Electrical)",
  "exportedEnergyOutput_MJ": "EE [MJ]",
    
  "traciGWP_kgCO2e": 8,
  "traciODP_kgCFC11e": 12,
  "traciAP_kgSO2e": 6,
  "traciEP_kgNe": 7,
  "traciPOCP_kgO3e": 10,
  "traciNRPE_MJ": 17
}





pdf_files = ['epd4080.pdf','epd4081.pdf','epd4082.pdf','epd4083.pdf','epd4084.pdf',
             'epd4085.pdf','epd4086.pdf', 'epd4087.pdf', 'epd4088.pdf', 'epd4089.pdf', 'epd4090.pdf']

# headers_map = {}
count = 5
for header in headers:
    sheet.cell(row=1, column=count).value = header

#     headers_map[header] = count

    count += 1
    
file_tables = {}
for file in pdf_files:
    file_tables[file] = prepare_df(file)

current_row = 2

#for i in range(0, len(file_tables)):
for file_name in file_tables:
#     df = file_tables[file_name][0]
    print(file_name)
    table_1 = file_tables[file_name][0]
    table_2 = file_tables[file_name][1]
    table_3 = file_tables[file_name][2]
    base_added = False
    stages_counter = 2
    for stage in stages:
        if not base_added:
            sheet.cell(row=current_row, column=1).value = file_name
            #sheet.cell(row=current_row, column=2).value = epd_number(file_name)
            sheet.cell(row=current_row, column=2).value = 'NO NUMBER CHECK IT!'
            sheet.cell(row=current_row, column=3).value = '/mnt/data/Downloads/2021-10-11/Environdec/New/' + file_name
            base_added = True
        
        sheet.cell(row=current_row, column=4).value = stage
        header_counter = 0
        for header in headers:
            try:
                sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
                #sheet.column_dimensions[header].width = 40
                if header_counter < 7:
                    sheet.cell(row=current_row, column= 5 + header_counter).value = float(table_1[table_1.eq(columns[header]).any(1)][stages_counter].tolist()[0])
#                 sheet.cell(row=current_row, column= 5 + header_counter).value = df[df.eq(columns[header]).any(1)][stages_counter].tolist()[0]
#             sheet.cell(row=current_row, column= 4 + stages_counter).value = df[df.eq(header).any(1)][stages_counter].values[0]
#                 if 6 < header_counter < 26:
#                     sheet.cell(row=current_row, column= 5 + header_counter).value = float(table_2[table_2.eq(columns[header]).any(1)][stages_counter].tolist()[0])
#                 if  25 < header_counter :
#                     sheet.cell(row=current_row, column= 5 + header_counter).value = get_traci(file_name, columns[header], stage)
                if 6 < header_counter < 17:
                    sheet.cell(row=current_row, column= 5 + header_counter).value = float(table_2[table_2.eq(columns[header]).any(1)][stages_counter].tolist()[0])
                if 16 < header_counter:
                    sheet.cell(row=current_row, column= 5 + header_counter).value = float(table_3[table_3.eq(columns[header]).any(1)][stages_counter].tolist()[0])

            except:
                print(stages_counter)
                print(header)
                #print(df[df.eq(header).any(1)][stages_counter])
#             stages_counter = stages_counter + 1
            header_counter = header_counter + 1
        stages_counter = stages_counter + 1
        current_row = current_row + 1
        
    current_row = current_row + 1 # PRAZAN RED IZMEDJU
        
        
    
# print(file_tables)
book.save(filename)
book.close()

epd4080.pdf
2
exportedEnergyOutput_MJ
2
traciGWP_kgCO2e
2
traciODP_kgCFC11e
2
traciAP_kgSO2e
2
traciEP_kgNe
2
traciPOCP_kgO3e
2
traciNRPE_MJ
3
exportedEnergyOutput_MJ
3
traciGWP_kgCO2e
3
traciODP_kgCFC11e
3
traciAP_kgSO2e
3
traciEP_kgNe
3
traciPOCP_kgO3e
3
traciNRPE_MJ
4
exportedEnergyOutput_MJ
4
traciGWP_kgCO2e
4
traciODP_kgCFC11e
4
traciAP_kgSO2e
4
traciEP_kgNe
4
traciPOCP_kgO3e
4
traciNRPE_MJ
5
exportedEnergyOutput_MJ
5
traciGWP_kgCO2e
5
traciODP_kgCFC11e
5
traciAP_kgSO2e
5
traciEP_kgNe
5
traciPOCP_kgO3e
5
traciNRPE_MJ
6
exportedEnergyOutput_MJ
6
traciGWP_kgCO2e
6
traciODP_kgCFC11e
6
traciAP_kgSO2e
6
traciEP_kgNe
6
traciPOCP_kgO3e
6
traciNRPE_MJ
7
exportedEnergyOutput_MJ
7
traciGWP_kgCO2e
7
traciODP_kgCFC11e
7
traciAP_kgSO2e
7
traciEP_kgNe
7
traciPOCP_kgO3e
7
traciNRPE_MJ
8
exportedEnergyOutput_MJ
8
traciGWP_kgCO2e
8
traciODP_kgCFC11e
8
traciAP_kgSO2e
8
traciEP_kgNe
8
traciPOCP_kgO3e
8
traciNRPE_MJ
epd4081.pdf
2
exportedEnergyOutput_MJ
2
traciGWP_kgCO2e
2
traciODP_kgCFC11e
2
traciAP

4
exportedEnergyOutput_MJ
4
traciGWP_kgCO2e
4
traciODP_kgCFC11e
4
traciAP_kgSO2e
4
traciEP_kgNe
4
traciPOCP_kgO3e
4
traciNRPE_MJ
5
exportedEnergyOutput_MJ
5
traciGWP_kgCO2e
5
traciODP_kgCFC11e
5
traciAP_kgSO2e
5
traciEP_kgNe
5
traciPOCP_kgO3e
5
traciNRPE_MJ
6
exportedEnergyOutput_MJ
6
traciGWP_kgCO2e
6
traciODP_kgCFC11e
6
traciAP_kgSO2e
6
traciEP_kgNe
6
traciPOCP_kgO3e
6
traciNRPE_MJ
7
exportedEnergyOutput_MJ
7
traciGWP_kgCO2e
7
traciODP_kgCFC11e
7
traciAP_kgSO2e
7
traciEP_kgNe
7
traciPOCP_kgO3e
7
traciNRPE_MJ
8
exportedEnergyOutput_MJ
8
traciGWP_kgCO2e
8
traciODP_kgCFC11e
8
traciAP_kgSO2e
8
traciEP_kgNe
8
traciPOCP_kgO3e
8
traciNRPE_MJ
epd4090.pdf
2
exportedEnergyOutput_MJ
2
traciGWP_kgCO2e
2
traciODP_kgCFC11e
2
traciAP_kgSO2e
2
traciEP_kgNe
2
traciPOCP_kgO3e
2
traciNRPE_MJ
3
cleanWaterNetUse_m3
3
exportedEnergyOutput_MJ
3
traciGWP_kgCO2e
3
traciODP_kgCFC11e
3
traciAP_kgSO2e
3
traciEP_kgNe
3
traciPOCP_kgO3e
3
traciNRPE_MJ
4
exportedEnergyOutput_MJ
4
traciGWP_kgCO2e
4
traciODP_kgCFC11e

In [ ]:
file_tables()

In [4]:
tables = camelot.read_pdf('./4/epd4080.pdf', pages='10,11')
print(len(tables))

3


In [5]:
tables[0].df

,0,1,2,3,4,5,6,7,8
0,Impact \nCategory,Unit,A1-A3,A4,C1,C2,C3,C4,D
1,GWP - Fossil,kg CO2 eq,35.0,0.249,0.338,75.4E-3,0,115E-3,-12.2
2,GWP - Biogenic,kg CO2 eq,0.076,84.7E-6,0.003,6.60E-6,0,4.51E-4,-0.023
3,GWP - Luluc,kg CO2 eq,0.453,113.6E-6,0.003,2.65E-6,0,1.86E-6,-248E-3
4,GWP - Total,kg CO2 eq,35.6,0.249,0.045,0.009,0,0.002,-12.4
5,ODP,kg CFC-11 eq,3.32E-6,56.7E-9,9.56E-9,2.14E-9,0,227E-12,-950E-9
6,AP,mol H+ eq,0.275,0.004,0.002,28.0E-6,0,13.4E-6,-0.097
7,EP - Freshwater,kg P eq,0.012,15.0E-6,357E-6,5.57E-6,0,3.19E-5,-0.004
8,*EP - Freshwater,kg PO4 eq,0.037,45.8E-6,1.09E-3,1.71E-5,0,9.75E-5,-0.013
9,EP - Marine,kg N eq,0.037,891E-6,362E-6,54.2E-6,0,0.003,-0.012


In [6]:
tables[1].df

,0,1,2,3,4,5,6,7,8
0,Impact \nCategory,Unit,A1-A3,A4,C1,C2,C3,C4,D
1,PERE,MJ,166,0.039,0.890,0.015,0,0.042,-87.3
2,PERM,MJ,0,0,0,0,0,0,0
3,PERT,MJ,166,0.039,0.890,0.015,0,0.042,-87.3
4,PENRE,MJ,361,3.70,3.72,1.23,0,0.846,-116
5,PENRM,MJ,0,0,0,0,0,0,0
6,PENRT,MJ,361,3.70,3.72,1.23,0,0.846,-116
7,SM,kg,0,0,0,0,0,0,0
8,RSF,MJ,0,0,0,0,0,0,0
9,NRSF,MJ,0,0,0,0,0,0,0


In [9]:
tables[2].df

,0,1,2,3,4,5,6,7,8
0,Impact \nCategory,Unit,A1-A3,A4,C1,C2,C3,C4,D
1,HWD,kg,0.109,0,0,0,0,0,0
2,NHWD,kg,0.512,0,0,0,0,0,0
3,RWD,kg,0,0,0,0,0,0,0
4,CRU,kg,0,0,0,0,0,0,0
5,MFR,kg,0,0,0,0,0,0,0
6,MER,kg,0,0,0,0,0,0,0
7,EE (Electrical),MJ,0,0,0,0,0,0,0
8,EE (Thermal),MJ,0,0,0,0,0,0,0
9,Acronyms,"HWD: Hazardous waste disposed, NHWD: Non-...",,,,,,,


In [11]:
tables = camelot.read_pdf('./4/epd4080.pdf',flavor='stream', pages='1')
print(len(tables))

1


In [2]:
tables = camelot.read_pdf('./4/epd4085.pdf', pages='10,11')
print(len(tables))

3


In [3]:
tables[0].df

,0,1,2,3,4,5,6,7,8
0,Impact \nCategory,Unit,A1-A3,A4,C1,C2,C3,C4,D
1,GWP - Fossil,kg CO2 eq,9.70,0.029,0.039,0.009,0,0.002,-6.55
2,GWP - Biogenic,kg CO2 eq,0.019,10E-6,357E-6,7E-6,0,0.003,-0.013
3,GWP - Luluc,kg CO2 eq,0.186,13E-6,373E-6,3E-6,0,2E-6,-0.134
4,GWP - Total,kg CO2 eq,9.91,0.029,0.040,0.009,0,0.006,-6.69
5,ODP,kg CFC-11 eq,766E-9,6.54E-9,1.10E-9,2.14E-9,0,238E-12,-511E-9
6,AP,mol H+ eq,0.076,424E-6,257E-6,28.0E-6,0,14.0E-6,-0.052
7,EP - Freshwater,kg P eq,0.004,2E-6,41E-6,644E-9,0,685E-9,-0.002
8,*EP - Freshwa-\nter,kg PO4 eq,0.011,5.29E-6,126E-6,1.97E-6,0,2E-6,-0.007
9,EP - Marine,kg N eq,0.010,103E-6,41.8E-6,6.26E-6,0,9.8E-6,-0.007


In [4]:
tables[1].df

,0,1,2,3,4,5,6,7,8,9
0,Impact \nCategory,Unit,A1-A3,A4,A5,C1,C2,C3,C4,D
1,PERE,MJ,66.9,4.53E-3,103E-3,1.78E-3,0,1.71E-3,-47.0,-905E-6
2,PERM,MJ,0,0,0,0,0,0,0,0
3,PERT,MJ,66.9,4.53E-3,103E-3,1.78E-3,0,1.71E-3,-47.0,-905E-6
4,PENRE,MJ,95.6,427E-3,430E-3,141E-3,0,29.4E-3,-62.4,-0.069
5,PENRM,MJ,0,0,0,0,0,0,0,0
6,PENRT,MJ,95.6,427E-3,430E-3,141E-3,0,29.4E-3,-62.4,-0.069
7,SM,kg,0,0,0,0,0,0,0,0
8,RSF,MJ,0,0,0,0,0,0,0,0
9,NRSF,MJ,0,0,0,0,0,0,0,0


In [5]:
tables[2].df

,0,1,2,3,4,5,6,7,8,9
0,Impact \nCategory,Unit,A1-A3,A4,A5,C1,C2,C3,C4,D
1,HWD,kg,0.018,0,0,0,0,0,0,0
2,NHWD,kg,0.084,0,0,0,0,0,0,0
3,RWD,kg,0,0,0,0,0,0,0,0
4,CRU,kg,0,0,0,0,0,0,0,0
5,MFR,kg,0,0,0,0,0,0,0,0
6,MER,kg,0,0,0,0,0,0,0,0
7,EE (Electrical),MJ,0,0,0,0,0,0,0,0
8,EE (Thermal),MJ,0,0,0,0,0,0,0,0
9,Acronyms,"HWD: Hazardous waste disposed, NHWD: Non-...",,,,,,,,


In [6]:
tables[2].df[2]

0     A1-A3
1     0.018
2     0.084
3         0
4         0
5         0
6         0
7         0
8         0
9          
10         
Name: 2, dtype: object

In [7]:
tables[2].df[2][1]

'0.018'

In [8]:
float(tables[2].df[2][1])

0.018

In [9]:
scientific_notation = "{:.2e}".format(float(tables[2].df[2][1]))

In [10]:
scientific_notation

'1.80e-02'

In [11]:
a ="{:.2e}".format(66.9)
a

'6.69e+01'

In [ ]:
stages = ['A1-A3', 'A4', 'C1', 'C2', 'C3', 'C4', 'D']

# EXCEL FILE
filename = 'NSF_excel_test.xlsx'
book = openpyxl.Workbook()
book.save(filename)


sheet = book['Sheet']
sheet.cell(row=1, column=1).value = 'Filename'
sheet.cell(row=1, column=2).value = 'EPD_Number'
sheet.cell(row=1, column=3).value = 'DownloadLink'
sheet.cell(row=1, column=4).value = 'Stages'


headers = [
    "impactGWP100_kgCO2e",
    "impactODP_kgCFC11e",
    "impactAP_kgSO2e",
    "impactEP_kgPO4e",
    "impactPOCP_kgEthenee",
    "impactADPElements_kgSbe",
    "impactADPFossilFuels_MJ",
    
    "renewablesUsedAsEnergy_MJ",
    "renewablesUsedAsMaterial_MJ",
    "embodiedRenewableEnergy_MJ",
    "nonRenewablesUsedAsEnergy_MJ",
    "nonRenewablesUsedAsMaterial_MJ",
    "embodiedEnergy_MJ",
    
    "recyclingMaterialUse_kg",
    "renewableRecylingFuelUse_MJ",
    "nonRenewableRecylingFuelUse_MJ",
    "cleanWaterNetUse_m3",
    "wasteHazardous_kg",
    "wasteNonHazardous_kg",
    "wasteRadioactive_kg",
    "reusableMaterialsOutput_kg",
    "recyclableMaterialsOutput_kg",
    "energyMaterialsOutput_kg",
    "exportedEnergyOutput_MJ_thermal",
    "exportedEnergyOutput_MJ_electric",
    "exportedEnergyOutput_MJ",
    
    "traciGWP_kgCO2e",
    "traciODP_kgCFC11e",
    "traciAP_kgSO2e",
    "traciEP_kgNe",
    "traciPOCP_kgO3e",
    "traciNRPE_MJ"
]

columns = {
  "impactGWP100_kgCO2e": ["GWP - Total"],
  "impactODP_kgCFC11e": ["ODP"],
  "impactAP_kgSO2e": ["AP"],
  "impactEP_kgPO4e": ["*EP - Freshwater"],
  "impactPOCP_kgEthenee": ["POCP"],
  "impactADPElements_kgSbe": ["ADPE"],
  "impactADPFossilFuels_MJ": ["ADPF"],
    
  "renewablesUsedAsEnergy_MJ": ["PERE"],
  "renewablesUsedAsMaterial_MJ": ["PERM"],
  "embodiedRenewableEnergy_MJ": ["PERT"],
  "nonRenewablesUsedAsEnergy_MJ": ["PENRE"],
  "nonRenewablesUsedAsMaterial_MJ": ["PENRM"],
  "embodiedEnergy_MJ": ["PENRT"],
  "recyclingMaterialUse_kg": ["SM"],
  "renewableRecylingFuelUse_MJ": ["RSF"],
  "nonRenewableRecylingFuelUse_MJ": ["NRSF"],
  "cleanWaterNetUse_m3": ["FW"],
    
  "wasteHazardous_kg": ["HWD"],
  "wasteNonHazardous_kg": ["NHWD"],
  "wasteRadioactive_kg": ["RWD"],
  "reusableMaterialsOutput_kg": ["CRU"],
  "recyclableMaterialsOutput_kg": ["MFR"],
  "energyMaterialsOutput_kg": ["MER"],
  "exportedEnergyOutput_MJ_thermal": ["EE (Thermal)"],
  "exportedEnergyOutput_MJ_electric": ["EE (Electrical)"],
  "exportedEnergyOutput_MJ": ["EE [MJ]"],
    
  "traciGWP_kgCO2e": 8,
  "traciODP_kgCFC11e": 12,
  "traciAP_kgSO2e": 6,
  "traciEP_kgNe": 7,
  "traciPOCP_kgO3e": 10,
  "traciNRPE_MJ": 17
}





pdf_files = ['epd4080.pdf','epd4081.pdf','epd4082.pdf','epd4083.pdf','epd4084.pdf',
             'epd4085.pdf','epd4086.pdf', 'epd4087.pdf', 'epd4088.pdf', 'epd4089.pdf', 'epd4090.pdf']